In [1]:
import amici
import os 

In [2]:
biomodel_id = 'BIOMD0000000630'
model_fp = f'../biosimulator_processes/model_files/sbml/{biomodel_id}_url.xml'
sbml_importer = amici.SbmlImporter(model_fp)

In [3]:
model_name = biomodel_id
model_dir = "model_dir"
sbml_importer.sbml2amici(model_name, model_dir)

In [12]:
# load the model module
model_module = amici.import_model_module(model_name, model_dir)
# instantiate model
model = model_module.getModel()
# instantiate solver
method = model.getSolver()

In [13]:
dir(model)

['__bool__',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__deref__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'clone',
 'fdsigmaydy',
 'fdspline_slopesdp',
 'fdspline_valuesdp',
 'fdtotal_cldp',
 'fdtotal_cldx_rdata',
 'fdx_rdatadp',
 'fdx_rdatadtcl',
 'fdx_rdatadx_solver',
 'get',
 'getAddSigmaResiduals',
 'getAlwaysCheckFinite',
 'getAmiciCommit',
 'getAmiciVersion',
 'getExpressionIds',
 'getExpressionNames',
 'getFixedParameterById',
 'getFixedParameterByName',
 'getFixedParameterIds',
 'getFixedParameterNames',
 'getFixedParameters',
 'getInitialStateSensitivities',
 'getInitialStates',
 'getMinimumSigmaResiduals',
 'getName',
 'getObservableIds

In [14]:
floating_species_list = list(model.getStateNames())

floating_species_list

['plasminogen',
 'plasmin',
 'single intact chain urokinase-type plasminogen activator',
 'two-chain urokinase-type plasminogen activator',
 'x',
 'x-plasmin']

In [18]:
# get/set parameter:

model.getParameterNames()

('degradation constant 1',
 'degradation constant 2',
 'hill cooperativity',
 'k1',
 'parameter_8',
 'k1',
 'k2',
 'k1',
 'k1',
 'v',
 'v',
 'v')

In [19]:
# set timepoints
model.setTimepoints([0, 1])

In [20]:
rdata = amici.runAmiciSimulation(model, method)

In [21]:
rdata.x

array([[1.00000000e-02, 0.00000000e+00, 2.00000000e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [1.95249026e-02, 1.82020113e-06, 4.90814799e-03, 1.32035442e-13,
        0.00000000e+00, 0.00000000e+00]])

In [ ]:
model.getStateNames()